In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#View Data
read=pd.read_csv("C:\\Users\\augusto\\.vscode\\Data Sci and Ml\\Linear_Regression\\Linear_regression+K-neigh\\Mall_Customers.csv",encoding='utf-8')
read.info()
read.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
5,6,Female,22,17,76
6,7,Female,35,18,6
7,8,Female,23,18,94
8,9,Male,64,19,3
9,10,Female,30,19,72


In [ ]:
#1.Cleaninng and Converting Score,Gender,Id (let's suppose that exists +200 y old people in this Analysis lol)

#Convert Wrong Data
read['CustomerID']=pd.to_numeric(read['CustomerID'],errors='coerce')
read['Age']=pd.to_numeric(read['Age'],errors='coerce')
read['Annual Income (k$)']=pd.to_numeric(read['Annual Income (k$)'],errors='coerce')
read['Spending Score (1-100)']=pd.to_numeric(read['Spending Score (1-100)'],errors='coerce')

#Convert Outliers
for x in read.index:
    if read.loc[x,'Spending Score (1-100)'] >100 or read.loc[x,'Spending Score (1-100)']<0:
        read.loc[x,'Spending Score (1-100)']=read['Spending Score (1-100)'].mean()

read.info()

In [ ]:
#3.Data
from math import floor
plt.figure(figsize=(20,12),facecolor='#E0FFFF',edgecolor='white')


#Correlation
corr=read[['Spending Score (1-100)','Age','Annual Income (k$)',]].corr()
dta=pd.DataFrame(corr)
print(dta)


#1.How old are the customers?
ages=pd.Series({"Median":read['Age'].median(),"Most Appeared":list(read['Age'].mode())[-2:],"Mean":read['Age'].mean()})
print(ages)

plt.subplot(2,2,1)
plt.hist(read['Age'],bins=40,color="skyblue",label=f'Std Deviation: {read['Age'].diff().sum()/len(read['Age'])}')
plt.grid(axis="both",color="blue",alpha=0.2)
plt.axvline(x=ages.loc['Mean'],color='b',linewidth=3,linestyle='-.',label=f"Mean : {ages.loc['Mean']}")
plt.axvline(x=ages.loc['Median'],color='purple',linewidth=3,linestyle='-.',label=f"Median : {ages.loc['Median']}")
plt.axvline(x=ages.loc['Most Appeared'],color='cyan',linewidth=3,linestyle='-.',label=f'Modal : {ages.loc['Most Appeared']}')
plt.legend()
plt.xlabel("Ages")
plt.ylabel("Num of people")

#2.Genre Spend
ml=0
fl=0
clmap=[]
for x in read.index:

        
    if read.loc[x,'Gender',]=='Male':
        ml+=(int(read.loc[x,'Annual Income (k$)']))
        clmap.append('b')

    elif read.loc[x,'Gender',]=='Female':
        fl+=(int(read.loc[x,'Annual Income (k$)']))
        clmap.append('r')


gen=np.array([ml,fl])
ds=[f'Male : {(ml/(ml+fl))*100:.2f}%',f'Female : {(fl/(ml+fl))*100:.2f}%']

plt.subplot(2,2,2)
plt.pie(gen,labels=ds,colors=['#00BFFF','#FF1493'],shadow=True)
plt.legend(title='Genre % of total Income')


#3.scatter between age/spend + gender color
plt.subplot(2,2,3)
plt.scatter(read['Age'],read['Annual Income (k$)'],alpha=0.5,c=clmap,s=50)
plt.xlabel('Ages/Gen with Color')
plt.ylabel('Income (k$)')
plt.legend()
plt.show()



In [4]:
#4.Model Predict Score

from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

#Test compare: Age/Income --> spendscore

model1=linear_model.LinearRegression()
model2=KNeighborsRegressor(n_neighbors=100)

x=np.c_[read["Age"],read['Annual Income (k$)']] 
y=np.c_[read['Spending Score (1-100)']]

#Training
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.25)

model1.fit(x_train,y_train)
model2.fit(x_train,y_train)


#Predicts
test1=model2.predict([[35,23]])
test2=model1.predict([[35,23]])

#We are using 2 parameters on x,so the predict will be an aproximate of the value

print(f'Linear Regression : {test1} , K-Nearest Neighbors : {test2} ') # --> Score 


#Tests/Accuracy 

from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error
from sklearn.model_selection import cross_val_score

acc1=model1.predict(x_test)
acc2=model2.predict(x_test)

print(f'''Linear Regression: 
      R2 score: {r2_score(y_test,acc1)} 
      MAE: {mean_absolute_error(y_test,acc1)} 
      RMSE: {root_mean_squared_error(y_test,acc1)}''')

print(f'''K-Neighbors:
      R2 score: {r2_score(y_test,acc2)} 
      R2 with Cross Validation: {sum(cross_val_score(KNeighborsRegressor(),x_test,y_test,cv=4,scoring='r2'))/4}
      MAE: {mean_absolute_error(y_test,acc2)} 
      RMSE: {root_mean_squared_error(y_test,acc2)}
      ''' )



Linear Regression : [[50.12]] , K-Nearest Neighbors : [[51.83417657]] 
Linear Regression: 
      R2 score: 0.14178491815332728 
      MAE: 18.8434966913734 
      RMSE: 22.48238074979407
K-Neighbors:
      R2 score: -0.02218776848008952 
      R2 with Cross Validation: -0.14992400856036103
      MAE: 19.8584 
      RMSE: 24.536327924121
      
